In [ ]:
# Import required libraries
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

# Define the directory path of the ear dataset
DATASET_DIR = '/path/to/ear/dataset'

# Define the number of classes (different individuals) in the dataset
NUM_CLASSES = 10

# Define the size of the input image
IMG_SIZE = (128, 128)

# Define the batch size and number of epochs for training
BATCH_SIZE = 32
EPOCHS = 20

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load the dataset and preprocess the images
data = []
labels = []
for i, folder_name in enumerate(os.listdir(DATASET_DIR)):
    for filename in os.listdir(os.path.join(DATASET_DIR, folder_name)):
        img_path = os.path.join(DATASET_DIR, folder_name, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, IMG_SIZE)
        data.append(img)
        labels.append(i)
data = np.array(data, dtype='float32')
data /= 255
data = np.reshape(data, (data.shape[0], IMG_SIZE[0], IMG_SIZE[1], 1))
labels = np_utils.to_categorical(labels, NUM_CLASSES)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_test, y_test))

# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')
